<a href="https://colab.research.google.com/github/contatofe/Estudos-e-Desafios/blob/main/Vis%C3%A3o%20Computacional%20-%20IA%20Expert/Transfer%C3%AAncia%20de%20aprendizagem%20-%20AI%20Expert/Transfer%C3%AAncia_de_aprendizagem_Classifica%C3%A7%C3%A3o_de_imagens_IA_Expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os
from google.colab import drive

import cv2
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#unzipando as imagens
path = '/content/drive/MyDrive/Arquivos - Dados/Curso - visão computacional/Datasets/cat_dog_2.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [4]:
#Importando dados de treino e teste

img_gen = ImageDataGenerator(rescale = 1./255,
                               rotation_range = 7,
                               horizontal_flip = True,
                               zoom_range = 0.2
                               )

dataset_train = img_gen.flow_from_directory('/content/cat_dog_2/training_set',
                                              target_size = (128, 128),
                                              batch_size = 32,
                                              class_mode = 'categorical',
                                              shuffle = True
                                              )
dataset_test = img_gen.flow_from_directory('/content/cat_dog_2/test_set',
                                              target_size = (128, 128),
                                              batch_size = 1,
                                              class_mode = 'categorical',
                                              shuffle = False
                                              )

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
# Importando modelo MobileNet

base_model = tf.keras.applications.MobileNetV2(input_shape = (128, 128, 3),
                                               include_top = False,
                                               weights = 'imagenet'
                                               )

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Congelando camadas já treinadas

for layer in base_model.layers:
  layer.trainable = False

In [7]:
# Camada densa personalizada

head_model = base_model.output
head_model = GlobalAveragePooling2D()(head_model)
head_model = Dense(641, activation = 'relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(641, activation = 'relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(2, activation = 'softmax')(head_model)

In [8]:
# Criando a Rede Neural

network = Model(inputs = base_model.input, outputs = head_model)
network.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = network.fit(dataset_train, epochs = 50)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 33s 165ms/step - accuracy: 0.8844 - loss: 0.2715
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 152ms/step - accuracy: 0.9566 - loss: 0.1091
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 162ms/step - accuracy: 0.9673 - loss: 0.0907
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.9650 - loss: 0.0899
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 164ms/step - accuracy: 0.9691 - loss: 0.0695
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.9784 - loss: 0.0608
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 158ms/step - accuracy: 0.9650 - loss: 0.0855
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.9771 - loss: 0.0664
Epoch 9/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 156ms/step - accuracy: 0.9697 - loss: 0.0731
Epoch 10/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 151ms/step - accuracy: 0.9796 - loss: 0.0508
Epoch 11/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 151ms/step - accuracy: 0.9820 - loss: 0.0512
Epoch 12/50
125/125 ━━━━━━━━━━

In [9]:
# Avaliação do modelo

network.evaluate(dataset_test)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9355 - loss: 0.2712


[0.2306179702281952, 0.9490000009536743]